# Data-Set Overview Plots

In [ ]:
using QAOA, Distributions, Interpolations
using HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

using PyCall
np = pyimport("numpy")

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

## Small-/Large-gap data comparison

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

# N = 17
# pattern = r"random_SK_instance_N_17_seed_(\d+)\.h5"

# N = 19
# pattern = r"random_SK_instance_N_19_seed_(\d+)\.h5"

pattern_dict = Dict()

### Mean-Field

In [ ]:
npts = 2048
coarse_times = range(0, 1, npts + 1)

exact_times = range(0, 1, 33);

In [ ]:
subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
filter!(x -> !occursin("results", x), instance_names)
filter!(x -> !occursin("undecided", x), instance_names);

In [ ]:
most_frustrated_spins = Dict(zip(h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6))),
                                      h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)))));

In [ ]:
ordered_seeds = []

mean_fields = Dict()
mean_scaled_flucs = Dict()
most_frustrated_flucs = Dict()

all_Hs = Dict()
all_eigenvals = Dict()
all_eigenstates = Dict()

# for instance_name in instance_names[310:320] # instance "23320"
for (k, instance_name) in enumerate(instance_names[1:1000])
    seed = match(pattern, instance_name)[1]    
    print(k, ", ")

    # eigenvalues and -vectors
    λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
    all_eigvecs = h5read(folder_name * instance_name, "exact_ARPACK_LM_lowest_eigvecs")

    J_mat = h5read(folder_name * instance_name, "J")
    mf_problem = Problem(0, J_mat)

    # mean-field solutions
    T_final = 32768.
    tol = 1e-6
    mf_sol = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_sol_T_final_%.0f_tol_1e%.0f", T_final, log10(tol)))
    sigma_star = sign.(mf_sol)
    h = mf_problem.local_fields
    J = mf_problem.couplings
    E_star = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1]) 
    
    if isapprox(E_star, λ[1, end], atol=1e-5) || isapprox(E_star, λ[2, end], atol=1e-2)
        continue
    end
        
    sol_t = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", T_final, log10(tol)))
    sol_u = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", T_final, log10(tol)))

    component_dict = Dict("x" => 1, "y" => 2, "z" => 3)
    nx_vals = reduce(hcat, [sol_u[k, component_dict["x"], :] for k in 1:size(sol_u)[1]])
    ny_vals = reduce(hcat, [sol_u[k, component_dict["y"], :] for k in 1:size(sol_u)[1]])
    nz_vals = reduce(hcat, [sol_u[k, component_dict["z"], :] for k in 1:size(sol_u)[1]])

    nx_coarse = reduce(hcat, [map(linear_interpolation(sol_t, nx_vals[spin_nr, :], extrapolation_bc=Line()), T_final .* coarse_times) for spin_nr in 1:N-1]) |> transpose
    ny_coarse = reduce(hcat, [map(linear_interpolation(sol_t, ny_vals[spin_nr, :], extrapolation_bc=Line()), T_final .* coarse_times) for spin_nr in 1:N-1]) |> transpose
    nz_coarse = reduce(hcat, [map(linear_interpolation(sol_t, nz_vals[spin_nr, :], extrapolation_bc=Line()), T_final .* coarse_times) for spin_nr in 1:N-1]) |> transpose

    mean_fields[seed] = [nx_coarse, ny_coarse, nz_coarse]

    # fluctuations
    T_final = 32000
    tol = 1e-8
    all_flucs = h5read(folder_name * "results_" * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
    lyapunov_exponent = sum(all_flucs, dims=1)
    if sum(lyapunov_exponent) |> abs < 1e4 # discard non-converged ones
        scale_factors = [1 .+ (nx_coarse[i, :] ./ (1 .+ sign(nz_coarse[i, end]) .* nz_coarse[i, :])).^2 for i in 1:N-1]
        # mean_scaled_flucs[seed] = mean([scale_factors[i] .* real(sqrt.(smoothen(all_flucs[i, :] .+ 0.0im, coarse_times))) for i in 1:N-1], dims=1)[1]
        mean_scaled_flucs[seed] = mean([scale_factors[i] .* real(sqrt.(all_flucs[i, :])) for i in 1:N-1], dims=1)[1]
        most_frustrated_flucs[seed] = all_flucs[most_frustrated_spins[seed], :]
    end

    # adiabatic theorem...
    H_x = SpinFluctuations.hamiltonian(1, 0, mf_problem.local_fields, mf_problem.couplings)
    H_z = SpinFluctuations.hamiltonian(0, 1, mf_problem.local_fields, mf_problem.couplings)
    all_Hs[seed] = [H_x, H_z]


    gs = [all_eigvecs[k, :, 1] for k in 1:length(exact_times)]
    first_ex = [all_eigvecs[k, :, 2] for k in 1:length(exact_times)]
    second_ex = [all_eigvecs[k, :, 3] for k in 1:length(exact_times)]
    third_ex = [all_eigvecs[k, :, 4] for k in 1:length(exact_times)]
    fourth_ex = [all_eigvecs[k, :, 5] for k in 1:length(exact_times)]

    all_eigenstates[seed] = [gs, first_ex, second_ex, third_ex, fourth_ex]
    all_eigenvals[seed] = [λ[1, :], λ[2, :], λ[3, :], λ[4, :], λ[5, :]]

    push!(ordered_seeds, seed)
end

In [ ]:
ordered_seeds |> size

## Data evaluation

In [ ]:
minigap_locs = Dict()
all_adiabatic_fracs = Dict()

all_frustrated_flucs = Dict()

for seed in ordered_seeds
    try
        nx_coarse, ny_coarse, nz_coarse = mean_fields[seed]

        top_idx = most_frustrated_spins[seed]
        scale_factor = 1 .+ (nx_coarse[top_idx, :] ./ (1 .+ sign(nz_coarse[top_idx, end]) .* nz_coarse[top_idx, :])).^2
        # all_frustrated_flucs[seed] = scale_factor .* (smoothen(most_frustrated_flucs[seed] .+ 0.0im, coarse_times) .|> sqrt |> real)
        all_frustrated_flucs[seed] = scale_factor .* (most_frustrated_flucs[seed] .|> sqrt |> real)

        H_x, H_z = all_Hs[seed]
        gs, first_ex, second_ex, third_ex, fourth_ex = all_eigenstates[seed]
        λ_1, λ_2, λ_3, λ_4, λ_5 = all_eigenvals[seed]

        mingap = minimum(λ_2 .- λ_1)
        gap_idx = findfirst(x -> x == mingap, λ_2 .- λ_1) 
        gaploc = exact_times[gap_idx]
        minigap_locs[seed] = gaploc

        overlap_01_x = [first_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
        overlap_01_z = [first_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

        overlap_02_x = [second_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
        overlap_02_z = [second_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

        overlap_03_x = [third_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
        overlap_03_z = [third_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

        overlap_04_x = [fourth_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
        overlap_04_z = [fourth_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]        

        frac_1 = abs.(overlap_01_z .- overlap_01_x) ./ (λ_2 .- λ_1)
        frac_2 = abs.(overlap_02_z .- overlap_02_x) ./ (λ_3 .- λ_1)
        frac_3 = abs.(overlap_03_z .- overlap_03_x) ./ (λ_4 .- λ_1)
        frac_4 = abs.(overlap_04_z .- overlap_04_x) ./ (λ_5 .- λ_1)
        
        all_adiabatic_fracs[seed] = [frac_1, frac_2, frac_3, frac_4]
    catch
        println(seed)
    end
end

## Plots

In [ ]:
ordered_seeds[1]

In [ ]:
for plot_seed in ordered_seeds
    # plot_seed = ordered_seeds[20]
    println(plot_seed)
    frac_1, frac_2, frac_3, frac_4 = all_adiabatic_fracs[plot_seed]
    frustrated_flucs = all_frustrated_flucs[plot_seed];

    figure(figsize=(4, 4))
    title(plot_seed)
    ax = subplot(211)
    plot(exact_times, frac_1, label="\$\\alpha = 0\$")
    plot(exact_times, frac_2, label="\$\\alpha = 1\$", lw=1.5)
    plot(exact_times, frac_3, label="\$\\alpha = 2\$", lw=2)
    plot(exact_times, frac_4, label="\$\\alpha = 3\$", lw=2.5)
    plot(exact_times, frac_1 .+ frac_2 .+ frac_3 .+ frac_4, "-k", lw=2.5)
    xlim(0., 1.)
    ylim(0, 5)
    ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
    ax.set_xticklabels([])
    axvline(minigap_locs[plot_seed], ls="--", c="k", alpha=0.5)
    legend(frameon=false)

    ax = subplot(212)
    plot(coarse_times,  frustrated_flucs .- mean_scaled_flucs[plot_seed], label=plot_seed)
    plot(coarse_times,  frustrated_flucs)
    xlim(0., 1.)
    ylim(0, 1.)
    xlabel("\$s\$")
    legend(frameon=false)

    tight_layout()
    savefig(("test_plots/seed_" * plot_seed * ".pdf"))
end

In [ ]:
mean_frac_1 = mean([seed_fracs[2][1] for seed_fracs in all_adiabatic_fracs])
mean_frac_2 = mean([seed_fracs[2][2] for seed_fracs in all_adiabatic_fracs])

# mean_frustrated_flucs = mean([(all_frustrated_flucs[seed] .- mean_scaled_flucs[seed]) ./ maximum(all_frustrated_flucs[seed] .- mean_scaled_flucs[seed]) for (seed, _) in all_adiabatic_fracs]);
# mean_frustrated_flucs = mean([(all_frustrated_flucs[seed] .- mean_scaled_flucs[seed]) for (seed, _) in all_adiabatic_fracs]);
mean_frustrated_flucs = mean([all_frustrated_flucs[seed] ./ maximum(all_frustrated_flucs[seed]) .- mean_scaled_flucs[seed] ./ maximum(mean_scaled_flucs[seed]) for (seed, _) in all_adiabatic_fracs]);

In [ ]:

figure(figsize=(4, 4))
ax = subplot(211)
plot(exact_times, mean_frac_1, label="\$\\alpha = 0\$")
plot(exact_times, mean_frac_2, label="\$\\alpha = 1\$", lw=2)
xlim(0., 1.)
ylim(0, 10)
ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
ax.set_xticklabels([])
legend(frameon=false)

ax = subplot(212)
plot(coarse_times,  mean_frustrated_flucs)
xlim(0., 1.)
ylim(0, )
xlabel("\$s\$")
tight_layout()